# imports

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk

# classes

In [ ]:
class Chessboard:
    def __init__(self):
        self.board = [
            ['r', 'n', 'b', 'q', 'k', 'b', 'n', 'r'],
            ['p', 'p', 'p', 'p', 'p', 'p', 'p', 'p'],
            [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
            [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
            [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
            [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
            ['P', 'P', 'P', 'P', 'P', 'P', 'P', 'P'],
            ['R', 'N', 'B', 'Q', 'K', 'B', 'N', 'R'],
        ]

        # Additional data structures to track piece movements and checkmate status
        self.piece_moved = [[False] * 8 for _ in range(8)]

    def display(self):
        for row in self.board:
            print(' '.join(str(piece) for piece in row))

    def move_piece(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        self.board[dest_row][dest_col] = self.board[src_row][src_col]
        self.board[src_row][src_col] = ' '

        # Update piece movement status
        self.piece_moved[src_row][src_col] = True
        self.piece_moved[dest_row][dest_col] = True

    def get_piece(self, row, col):
        return self.board[row][col]

    def get_piece_color(self, row, col):
        piece = self.get_piece(row, col)
        return 'white' if piece.isupper() else 'black'

    def is_valid_square(self, row, col):
        return 0 <= row < 8 and 0 <= col < 8

    def piece_has_moved(self, row, col):
        return self.piece_moved[row][col]

    def is_stalemate(self):
        # Check if it's a stalemate (no legal moves for the current player)
        for row in range(8):
            for col in range(8):
                piece = self.get_piece(row, col)
                if piece.isupper() == self.is_white_turn():
                    for dest_row in range(8):
                        for dest_col in range(8):
                            if self.is_valid_move((row, col), (dest_row, dest_col)):
                                return False
        return True
    
    def is_check(self, color):
        # Check if the king of the specified color is in check
        king_position = self.find_king(color)
        return self.is_square_under_attack(king_position[0], king_position[1], color)

    def is_checkmate(self, color):
        # Check if it's a checkmate for the specified color
        if not self.is_in_check(color):
            return False

        for row in range(8):
            for col in range(8):
                if self.get_piece_color(row, col) == color:
                    for dest_row in range(8):
                        for dest_col in range(8):
                            if self.is_valid_move((row, col), (dest_row, dest_col)):
                                # Simulate the move and check if the king is still in check
                                self.simulate_move((row, col), (dest_row, dest_col))
                                if not self.is_in_check(color):
                                    # The move removes the check, so it's not checkmate
                                    self.undo_move()
                                    return False
                                self.undo_move()

        return True

    def promote_pawn(self, dest_row, dest_col, promotion_piece):
        self.board[dest_row][dest_col] = promotion_piece

    def castle_kingside(self, color):
        if color == 'white':
            row = 7
        else:
            row = 0

        if not self.piece_has_moved(row, 4) and not self.piece_has_moved(row, 7):
            # Check if the squares between the king and rook are empty
            if all(self.get_piece(row, col) == ' ' for col in range(5, 7)):
                # Check if the squares the king crosses are not under attack
                if not any(self.is_square_under_attack(row, col, color) for col in range(4, 7)):
                    # Perform kingside castling
                    self.move_piece((row, 4), (row, 6))
                    self.move_piece((row, 7), (row, 5))
                    return True

        return False
    
    def castle_queenside(self, color):
        if color == 'white':
            row = 7
        else:
            row = 0

        if not self.piece_has_moved(row, 4) and not self.piece_has_moved(row, 0):
            # Check if the squares between the king and rook are empty
            if all(self.get_piece(row, col) == ' ' for col in range(1, 4)):
                # Check if the squares the king crosses are not under attack
                if not any(self.is_square_under_attack(row, col, color) for col in range(2, 5)):
                    # Perform queenside castling
                    self.move_piece((row, 4), (row, 2))
                    self.move_piece((row, 0), (row, 3))
                    return True

        return False

    def is_valid_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        piece = self.get_piece(src_row, src_col)

        if not self.is_valid_square(dest_row, dest_col):
            return False

        dest_piece_color = self.get_piece_color(dest_row, dest_col)

        if dest_piece_color == self.get_piece_color(src_row, src_col):
            return False

        # Check specific move rules for each piece type
        if piece.upper() == 'P':
            return self.is_valid_pawn_move(source, destination)
        elif piece.upper() == 'R':
            return self.is_valid_rook_move(source, destination)
        elif piece.upper() == 'N':
            return self.is_valid_knight_move(source, destination)
        elif piece.upper() == 'B':
            return self.is_valid_bishop_move(source, destination)
        elif piece.upper() == 'Q':
            return self.is_valid_queen_move(source, destination)
        elif piece.upper() == 'K':
            return self.is_valid_king_move(source, destination)

        return False
    
    def is_valid_pawn_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination
        color = self.get_piece_color(src_row, src_col)
        direction = 1 if color == 'white' else -1

        # Check if the destination is within the pawn's movement range
        if dest_col != src_col:
            # Pawn captures diagonally
            if abs(dest_col - src_col) == 1 and dest_row - src_row == direction:
                dest_piece = self.get_piece(dest_row, dest_col)
                return dest_piece and self.get_piece_color(dest_row, dest_col) != color
            else:
                return False
        else:
            # Pawn moves forward
            if dest_row - src_row == direction and self.get_piece(dest_row, dest_col) == ' ':
                return True
            # Pawn's first move allows for a two-square advance
            elif (
                not self.piece_has_moved(src_row, src_col)
                and dest_row - src_row == 2 * direction
                and self.get_piece(src_row + direction, dest_col) == ' '
                and self.get_piece(dest_row, dest_col) == ' '
            ):
                return True
            else:
                return False

    def is_valid_rook_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        # Check if the move is along a rank or file
        if src_row == dest_row:
            # Check if the path is clear horizontally
            step = 1 if dest_col > src_col else -1
            for col in range(src_col + step, dest_col, step):
                if self.get_piece(src_row, col) != ' ':
                    return False
            return True
        elif src_col == dest_col:
            # Check if the path is clear vertically
            step = 1 if dest_row > src_row else -1
            for row in range(src_row + step, dest_row, step):
                if self.get_piece(row, src_col) != ' ':
                    return False
            return True
        else:
            return False

    def is_valid_knight_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        # Check if the move is an L-shape (two squares in one direction and one square perpendicular)
        row_diff = abs(dest_row - src_row)
        col_diff = abs(dest_col - src_col)
        return (row_diff == 2 and col_diff == 1) or (row_diff == 1 and col_diff == 2)

    def is_valid_bishop_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        # Check if the move is along a diagonal
        if abs(dest_row - src_row) == abs(dest_col - src_col):
            # Check if the path is clear diagonally
            row_step = 1 if dest_row > src_row else -1
            col_step = 1 if dest_col > src_col else -1
            row, col = src_row + row_step, src_col + col_step
            while row != dest_row and col != dest_col:
                if self.get_piece(row, col) != ' ':
                    return False
                row += row_step
                col += col_step
            return True
        else:
            return False

    def is_valid_queen_move(self, source, destination):
        return self.is_valid_rook_move(source, destination) or self.is_valid_bishop_move(source, destination)

    def is_valid_king_move(self, source, destination):
        src_row, src_col = source
        dest_row, dest_col = destination

        # Check if the move is within one square in any direction
        return abs(dest_row - src_row) <= 1 and abs(dest_col - src_col) <= 1
    
    def is_square_under_attack(self, row, col, attacker_color):
        # Check if the square is under attack by any piece of the specified color
        for i in range(8):
            for j in range(8):
                piece = self.get_piece(i, j)
                if piece.isupper() == (attacker_color == 'white') and self.is_valid_move((i, j), (row, col)):
                    return True
        return False
    
    def is_in_check(self, color):
        # Check if the king of the specified color is in check
        king_position = self.find_king(color)
        return self.is_square_under_attack(king_position[0], king_position[1], color)
    
    def find_king(self, color):
        # Find the position of the king of the specified color
        for row in range(8):
            for col in range(8):
                if self.get_piece(row, col).upper() == 'K' and self.get_piece_color(row, col) == color:
                    return row, col
        return None
    
    def simulate_move(self, source, destination):
        # Simulate a move on the board for validation purposes
        src_piece = self.get_piece(source[0], source[1])
        dest_piece = self.get_piece(destination[0], destination[1])

        # Make the move on the board
        self.move_piece(source, destination)

        # Check if the move puts the own king in check
        if self.is_in_check(self.get_piece_color(destination[0], destination[1])):
            self.undo_move()
            raise ValueError("Move puts own king in check")

        return src_piece, dest_piece

    def undo_move(self, source, destination, src_piece, dest_piece):
        # Undo a previously simulated move on the board
        self.move_piece(destination, source)
        self.board[destination[0]][destination[1]] = dest_piece
        self.board[source[0]][source[1]] = src_piece

In [ ]:
class ChessGame:
    def __init__(self):
        self.board = Chessboard()
        self.current_turn = 'white'

    def play(self):
        while not self.is_game_over():
            self.display_board()
            print(f"\nIt's {self.current_turn}'s turn.")
            move_successful = False
            while not move_successful:
                move = input("Enter your move (e.g., 'e2 e4'): ")
                move_successful = self.make_move(move)
                if not move_successful:
                    print("Invalid move. Try again.")

            self.switch_turn()

        print("Game over.")

    def is_game_over(self):
        return self.board.is_stalemate() or self.board.is_checkmate(self.current_turn)
    
    def is_current_player_piece(self, source):
        # Check if the piece at the source position belongs to the current player
        src_row, src_col = source
        piece_color = self.board.get_piece_color(src_row, src_col)
        return piece_color == self.current_turn

    def make_move(self, source, destination):
        move = f"{chr(source[1] + ord('a'))}{source[0] + 1} {chr(destination[1] + ord('a'))}{destination[0] + 1}"

        if not self.validate_move(source, destination):
            return False

        piece = self.board.get_piece(source[0], source[1])
        self.board.move_piece(source, destination)

        if piece.upper() == 'P' and destination[0] in [0, 7]:
            self.handle_pawn_promotion(destination)

        self.handle_castling(piece, source[0], destination[1])
        self.switch_turn()  # Switch turns here
        return True

    def handle_pawn_promotion(self, destination):
        dest_row, dest_col = destination
        promotion_piece = input(f"Choose a piece for promotion (Q, R, N, B): ").upper()
        self.board.promote_pawn(dest_row, dest_col, promotion_piece)

    def handle_castling(self, piece, src_row, dest_col):
        if piece.upper() == 'K' and dest_col - src_row == 2:
            self.board.castle_kingside(self.current_turn)
        elif piece.upper() == 'K' and dest_col - src_row == -2:
            self.board.castle_queenside(self.current_turn)

    def switch_turn(self):
        self.current_turn = 'black' if self.current_turn == 'white' else 'white'

    def parse_move(self, move):
        source_col, source_row, dest_col, dest_row = ord(move[0]) - ord('a'), int(move[1]) - 1, ord(move[3]) - ord('a'), int(move[4]) - 1
        return (source_row, source_col), (dest_row, dest_col)

    def validate_move(self, source, destination):
        if not self.board.is_valid_square(*source):
            print("Invalid source square.")
            return False

        if not self.board.is_valid_square(*destination):
            print("Invalid destination square.")
            return False

        piece_color = self.board.get_piece_color(*source)
        if piece_color != self.current_turn:
            print("It's not your turn.")
            return False

        if not self.board.is_valid_move(source, destination):
            print("Invalid move for the selected piece.")
            return False

        return True

    def display_board(self):
        self.board.display()

In [ ]:
class ChessGUI(tk.Tk):
    def __init__(self, game, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        self.game = game
        self.canvas = tk.Canvas(self, width=400, height=400)
        self.canvas.pack()

        self.cell_size = 50  # Adjust the cell size as needed
        self.piece_images = self.load_piece_images()
        self.draw_chessboard()
        self.draw_pieces()

        # Bind mouse events to handle user input
        self.canvas.bind("<Button-1>", self.on_click)

        self.selected_piece = None  # Store the selected piece

        self.turn_status_label = tk.Label(self, text=f"Turn: {self.game.current_turn}")
        self.turn_status_label.pack()

    def load_piece_images(self):
        # Load chess piece images and resize them
        piece_images = {
            'P': self.resize_image(Image.open('chess_assets/whitepawn.png')),
            'R': self.resize_image(Image.open('chess_assets/whiterook.png')),
            'N': self.resize_image(Image.open('chess_assets/whiteknight.png')),
            'B': self.resize_image(Image.open('chess_assets/whitebishop.png')),
            'Q': self.resize_image(Image.open('chess_assets/whitequeen.png')),
            'K': self.resize_image(Image.open('chess_assets/whiteking.png')),
            'p': self.resize_image(Image.open('chess_assets/blackpawn.png')),
            'r': self.resize_image(Image.open('chess_assets/blackrook.png')),
            'n': self.resize_image(Image.open('chess_assets/blackknight.png')),
            'b': self.resize_image(Image.open('chess_assets/blackbishop.png')),
            'q': self.resize_image(Image.open('chess_assets/blackqueen.png')),
            'k': self.resize_image(Image.open('chess_assets/blackking.png')),
        }
        return piece_images

    def resize_image(self, img):
        # Resize the image to fit the cell size
        return ImageTk.PhotoImage(img.resize((self.cell_size, self.cell_size)))

    def draw_chessboard(self):
        for row in range(8):
            for col in range(8):
                x0, y0 = col * self.cell_size, row * self.cell_size
                x1, y1 = x0 + self.cell_size, y0 + self.cell_size
                color = "white" if (row + col) % 2 == 0 else "black"
                self.canvas.create_rectangle(x0, y0, x1, y1, fill=color)

    def draw_pieces(self):
        for row in range(8):
            for col in range(8):
                piece = self.game.board.board[row][col]
                if piece != ' ':
                    x, y = col * self.cell_size, row * self.cell_size
                    # Use create_image to display the piece image
                    self.canvas.create_image(x, y, anchor=tk.NW, image=self.piece_images[piece])

    def on_click(self, event):
        # Round the mouse coordinates to the nearest grid cell
        row = int(event.y / self.cell_size)
        col = int(event.x / self.cell_size)

        if self.selected_piece is None:
            # No piece is selected, so check if there is a piece at the clicked position
            piece = self.game.board.board[row][col]
            if piece != ' ' and self.game.is_current_player_piece((row, col)):
                self.selected_piece = (row, col)  # Store the selected piece
        else:
            # A piece is already selected, attempt to make the move
            source = self.selected_piece
            destination = (row, col)
            move = f"{chr(source[1] + ord('a'))}{source[0] + 1} {chr(destination[1] + ord('a'))}{destination[0] + 1}"
            if self.game.make_move(source, destination):
                # Move successful, update the GUI
                self.draw_chessboard()
                self.draw_pieces()
                self.update_turn_status()  # Update the turn status label
                self.check_game_status()  # Check for check, checkmate, or stalemate
            
            self.selected_piece = None  # Reset the selected piece

    def update_turn_status(self):
        # Update the turn status label
        self.turn_status_label.config(text=f"Turn: {self.game.current_turn}")

    def check_game_status(self):
        # Check for check, checkmate, or stalemate
        if self.game.board.is_check(self.game.current_turn):
            print("Check!")
            if self.game.is_checkmate():
                print("Checkmate!")
        elif self.game.is_stalemate():
            print("Stalemate!")

# playing chess

In [ ]:
if __name__ == "__main__":
    chess_game = ChessGame()
    chess_gui = ChessGUI(chess_game)
    chess_gui.mainloop()